# Loading the reference images

In [ ]:
import os
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt

from cloth_tools.dataset.format import load_competition_observation

data_dir = Path("data")
references_dir = data_dir / "cloth_competition_references_0002"
teams_dir = data_dir / "evaluation_icra_2024"

team_name = "dev_team"
team_dir = teams_dir / team_name

team_dir

In [ ]:
os.path.exists(references_dir)

In [ ]:
reference_sample_dirs = [references_dir / dirname for dirname in sorted(os.listdir(references_dir))]
reference_sample_dirs

In [ ]:
reference_observation_dirs = [sample_dir / "observation_result" for sample_dir in reference_sample_dirs]

# observation_dirs = [dataset_dir / ref_dir for ref_dir in sorted(os.listdir(dataset_dir))]
len(reference_observation_dirs)

In [ ]:
import json

reference_images = {}
reference_surface_areas = {}

for cloth_id, observation_dir in enumerate(reference_observation_dirs):
    observation = load_competition_observation(observation_dir)

    results_file = observation_dir / "result.json"

    if results_file.exists():
        with open(results_file, "r") as f:
            result = json.load(f)
        surface_area = result["coverage"]
        reference_images[f"Reference {cloth_id} ({surface_area:.3f} m2)"] = observation.image_left
        reference_surface_areas[cloth_id] = surface_area
    else:
        reference_images[f"Reference {cloth_id}"] = observation.image_left


In [ ]:
import matplotlib.pyplot as plt

def display_images(images_dict):
    """Displays reference images in a grid layout.

    Args:
        reference_images (dict): A dictionary where keys are cloth IDs and
                                 values are the corresponding images.
    """

    n_images = len(images_dict)
    cols = 4  # Adjust the number of columns as desired
    rows = (n_images // cols) + (n_images % cols > 0)  # Calculate rows for the grid

    fig, axes = plt.subplots(nrows=rows, ncols=cols, figsize=(cols * 4, rows * 3))  # Create subplots

    cloth_ids = list(images_dict.keys())
    images = list(images_dict.values())

    for i, ax in enumerate(axes.flat):
        if i < n_images:
            image = images[i]
            cloth_id = cloth_ids[i]
            ax.imshow(image)
            ax.set_title(f"{cloth_id}")
            ax.axis("off")  # Hide unused subplots
        # else:

    plt.tight_layout()
    plt.show()

# Example usage with your dictionary:
display_images(reference_images) 

In [ ]:
def sample_id_to_reference_id(sample_id):
    return sample_id % 8

sample_id_to_reference_id(8), sample_id_to_reference_id(10)

In [ ]:
team_sample_dirs = [team_dir / dirname for dirname in sorted(os.listdir(team_dir)) if dirname.startswith("sample_")]

team_sample_dirs_complete = []
team_result_observations = []

i = 0 
for team_sample_dir in team_sample_dirs:
    try:
        observation_results_dir = team_sample_dir / "observation_result"
        if not os.path.exists(observation_results_dir):
            continue 
        observation = load_competition_observation(observation_results_dir)   
        team_result_observations.append(observation)
        team_sample_dirs_complete.append(team_sample_dir)
    except:
        print(f"Failed to load result observation from {observation_results_dir}")
        continue


print(f"Loaded {len(team_result_observations)} observations from team {team_name}")

In [ ]:
team_surface_areas = {}
team_images = {}
team_coverages = {}

for i, observation in enumerate(team_result_observations):
    results_file = team_sample_dirs_complete[i] / "observation_result" / "result.json"
    if results_file.exists():
        with open(results_file, "r") as f:
            result = json.load(f)
        team_surface_area = result["coverage"]
        reference_id = sample_id_to_reference_id(i)
        reference_surface_area = reference_surface_areas[reference_id]
        coverage = team_surface_area / reference_surface_area
        team_surface_areas[i] = team_surface_area
        team_coverages[i] = coverage
        team_images[f"Evaluation {i} ({team_surface_area:.3f} m2, {coverage:.2f})"] = observation.image_left
    else:
        team_images[f"Evaluation {i}"] = observation.image_left
        print(f"Results file not found for observation {i}")

In [ ]:
display_images(team_images)

In [ ]:
average_coverage = np.mean(list(team_coverages.values()))

print(f"Average coverage: {average_coverage:.2f}")